##### Usecase:Get sales data by orderstatus
###### Solutions are given for following problem statements
###### Problem statement-1:Get sales data for each orderstatus
###### Problem statement-2:Get sales data for each orderstatus perday
###### Problem statement-3:Get total orders for each order status permonth
###### Problem statement-4:Get total orders for each order status peryear
###### Problem statement-5:Get monthly sales data for each order status for first quarter of 2014
###### Problem statement-6:Get monthly sales data for last quarter of 2013 with neither 'COMPLETE' nor 'CLOSED' orders
###### Problem statement-7:Get monthly sales data for last quarter of 2013 and first quarter of 2014  with 'CANCELED' orders
###### Problem statement-8:Get monthly sales data for 'PENDING' orders
###### Problem statement-9:Get monthly sales data for 'PROCESSING' OR ON_HOLD orders
###### Problem statement-10:Get monthly sales data with  'COMPLETE' AND 'CLOSED' orders

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum,round,avg,rank, col
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.functions import unix_timestamp, lit
spark = SparkSession.builder.master('local').appName('SalesDataForeachorderstatus').enableHiveSupport().getOrCreate()

In [3]:
ordersdf = spark.read.csv("/FileStore/tables/retaildbtext/Orders.txt", sep=',',schema ='order_id int,order_date string,order_customer_id int, order_status string')
ordersdf.show(5,False)

+--------+----------+-----------------+---------------+
order_id|order_date|order_customer_id|order_status |
+--------+----------+-----------------+---------------+
1 |2013-07-25|11599 |CLOSED |
2 |2013-07-25|256 |PENDING_PAYMENT|
3 |2013-07-25|12111 |COMPLETE |
4 |2013-07-25|8827 |CLOSED |
5 |2013-07-25|11318 |COMPLETE |
+--------+----------+-----------------+---------------+
only showing top 5 rows

In [4]:
orderItemsdf = spark.read.csv("/FileStore/tables/retaildbtext/Order_Items.txt", sep=',', schema='order_item_id int, order_item_order_id int, order_item_product_id  int, order_item_quantity int,order_item_subtotal float,order_item_product_price float')
orderItemsdf.show(5,False)

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
1 |1 |957 |1 |299.98 |299.98 |
2 |2 |1073 |1 |199.99 |199.99 |
3 |2 |502 |5 |250.0 |50.0 |
4 |2 |403 |1 |129.99 |129.99 |
5 |4 |897 |2 |49.98 |24.99 |
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
only showing top 5 rows

In [5]:
ordersMap = ordersdf.select('order_id','order_date','order_status')
ordersMap.show(5,False)

+--------+----------+---------------+
order_id|order_date|order_status |
+--------+----------+---------------+
1 |2013-07-25|CLOSED |
2 |2013-07-25|PENDING_PAYMENT|
3 |2013-07-25|COMPLETE |
4 |2013-07-25|CLOSED |
5 |2013-07-25|COMPLETE |
+--------+----------+---------------+
only showing top 5 rows

In [6]:
orderItemsMap=orderItemsdf.select('order_item_order_id','order_item_subtotal','order_item_quantity')
orderItemsMap.show(5,False)

+-------------------+-------------------+-------------------+
order_item_order_id|order_item_subtotal|order_item_quantity|
+-------------------+-------------------+-------------------+
1 |299.98 |1 |
2 |199.99 |1 |
2 |250.0 |5 |
2 |129.99 |1 |
4 |49.98 |2 |
+-------------------+-------------------+-------------------+
only showing top 5 rows

In [7]:
ordersdfjoin = ordersMap.join(orderItemsMap, ordersMap.order_id == orderItemsMap.order_item_order_id,'inner')
ordersdfjoin.show(5,False)

+--------+----------+---------------+-------------------+-------------------+-------------------+
order_id|order_date|order_status |order_item_order_id|order_item_subtotal|order_item_quantity|
+--------+----------+---------------+-------------------+-------------------+-------------------+
1 |2013-07-25|CLOSED |1 |299.98 |1 |
2 |2013-07-25|PENDING_PAYMENT|2 |199.99 |1 |
2 |2013-07-25|PENDING_PAYMENT|2 |250.0 |5 |
2 |2013-07-25|PENDING_PAYMENT|2 |129.99 |1 |
4 |2013-07-25|CLOSED |4 |49.98 |2 |
+--------+----------+---------------+-------------------+-------------------+-------------------+
only showing top 5 rows

In [8]:
ordersdfSalesmap = ordersdfjoin.select('order_id','order_date','order_status','order_item_subtotal','order_item_quantity')
ordersdfSalesmap.show(5,False)

+--------+----------+---------------+-------------------+-------------------+
order_id|order_date|order_status |order_item_subtotal|order_item_quantity|
+--------+----------+---------------+-------------------+-------------------+
1 |2013-07-25|CLOSED |299.98 |1 |
2 |2013-07-25|PENDING_PAYMENT|199.99 |1 |
2 |2013-07-25|PENDING_PAYMENT|250.0 |5 |
2 |2013-07-25|PENDING_PAYMENT|129.99 |1 |
4 |2013-07-25|CLOSED |49.98 |2 |
+--------+----------+---------------+-------------------+-------------------+
only showing top 5 rows

##### Problem statement-1:Get sales data for each orderstatus

In [10]:
SalesDataPerorderPerstatus = ordersdfSalesmap.groupBy('order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_perstatus")).sort('order_status')
SalesDataPerorderPerstatus.coalesce(1).write.csv("/FileStore/tables/SalesDataPerorderPerstatus", compression="none", header ='true')
SalesDataPerorderPerstatus.show(20,False)

+---------------+--------+---------+---------+-------------+-----------------------+-----------------------+
order_status |avgsales|min_sales|max_sales|Totalsales |Totalquantity_perstatus|Numberofsales_perstatus|
+---------------+--------+---------+---------+-------------+-----------------------+-----------------------+
CANCELED |197.79 |14.99 |599.99 |696031.0 |7702 |3519 |
CLOSED |200.13 |9.99 |1999.99 |3736048.86 |40510 |18668 |
COMPLETE |198.75 |9.99 |1999.99 |1.127693391E7|123873 |56740 |
ON_HOLD |198.95 |9.99 |1999.99 |1864731.28 |20414 |9373 |
PAYMENT_REVIEW |199.13 |9.99 |499.95 |357841.46 |4013 |1797 |
PENDING |199.67 |9.99 |999.99 |3851881.36 |42260 |19291 |
PENDING_PAYMENT|199.36 |9.99 |999.99 |7581671.2 |82935 |38031 |
PROCESSING |200.5 |9.99 |1999.99 |4190636.84 |45622 |20901 |
SUSPECTED_FRAUD|197.74 |14.99 |499.95 |766844.69 |8429 |3878 |
+---------------+--------+---------+---------+-------------+-----------------------+-----------------------+

##### Problem statement-2:Get sales data for each orderstatus perday

In [12]:
SalesDataPerstatusPerday = ordersdfSalesmap.groupBy(substring('order_date',0,10).alias("orderdate"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_perday")).sort(substring('order_date',0,10).alias("orderdate"),'order_status')
SalesDataPerstatusPerday.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPerday", compression="none", header ='true')
SalesDataPerstatusPerday.show(20,False)

+----------+---------------+--------+---------+---------+----------+-----------------------+--------------------+
orderdate |order_status |avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_perday|
+----------+---------------+--------+---------+---------+----------+-----------------------+--------------------+
2013-07-25|CANCELED |214.99 |129.99 |299.98 |429.97 |2 |2 |
2013-07-25|CLOSED |213.28 |49.98 |599.99 |11516.91 |108 |54 |
2013-07-25|COMPLETE |208.65 |39.99 |499.95 |20030.32 |222 |96 |
2013-07-25|ON_HOLD |158.32 |59.99 |299.98 |1899.84 |22 |12 |
2013-07-25|PAYMENT_REVIEW |236.62 |49.98 |499.95 |1419.74 |20 |6 |
2013-07-25|PENDING |203.64 |39.99 |399.98 |4887.46 |49 |24 |
2013-07-25|PENDING_PAYMENT|186.97 |19.98 |499.95 |17014.02 |181 |91 |
2013-07-25|PROCESSING |201.68 |49.98 |499.95 |10285.64 |131 |51 |
2013-07-25|SUSPECTED_FRAUD|223.31 |129.99 |299.98 |669.93 |6 |3 |
2013-07-26|CANCELED |166.23 |49.98 |399.98 |1329.85 |17 |8 |
2013-07-26|CLOSED |193.04 |30.0 |499.95 |12547.35 |152 |65 |
2013-07-26|COMPLETE |191.66 |29.98 |499.95 |42165.88 |481 |220 |
2013-07-26|ON_HOLD |183.0 |9.99 |399.98 |9149.88 |101 |50 |
2013-07-26|PAYMENT_REVIEW |158.64 |35.98 |299.95 |2855.59 |44 |18 |
2013-07-26|PENDING |201.45 |15.99 |399.98 |19741.87 |213 |98 |
2013-07-26|PENDING_PAYMENT|202.29 |31.99 |499.95 |31152.51 |340 |154 |
2013-07-26|PROCESSING |222.08 |39.99 |399.98 |15323.46 |145 |69 |
2013-07-26|SUSPECTED_FRAUD|187.82 |47.99 |399.98 |2253.78 |18 |12 |
2013-07-27|CLOSED |214.86 |17.99 |499.95 |15255.27 |161 |71 |
2013-07-27|COMPLETE |197.36 |27.99 |499.95 |33156.21 |382 |168 |
+----------+---------------+--------+---------+---------+----------+-----------------------+--------------------+
only showing top 20 rows

##### Problem statement-3:Get total orders for each order status permonth

In [14]:
SalesDataPerstatusPermonth = ordersdfSalesmap.groupBy(substring('order_date',0,7).alias("ordermonth"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_permonth")).sort(substring('order_date',0,7).alias("ordermonth"),'order_status')
SalesDataPerstatusPermonth.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPermonth", compression="none", header ='true')
SalesDataPerstatusPermonth.show(20,False)

+----------+---------------+--------+---------+---------+----------+-----------------------+----------------------+
ordermonth|order_status |avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_permonth|
+----------+---------------+--------+---------+---------+----------+-----------------------+----------------------+
2013-07 |CANCELED |170.39 |22.0 |399.98 |9882.41 |136 |58 |
2013-07 |CLOSED |204.47 |17.99 |599.99 |78107.44 |833 |382 |
2013-07 |COMPLETE |197.8 |17.99 |499.95 |255358.01 |2880 |1291 |
2013-07 |ON_HOLD |188.92 |9.99 |499.95 |37594.59 |435 |199 |
2013-07 |PAYMENT_REVIEW |180.18 |35.98 |499.95 |7747.87 |103 |43 |
2013-07 |PENDING |207.51 |15.99 |499.95 |86945.94 |953 |419 |
2013-07 |PENDING_PAYMENT|204.15 |19.98 |499.95 |175973.91 |1824 |862 |
2013-07 |PROCESSING |197.45 |19.99 |499.95 |97341.74 |1097 |493 |
2013-07 |SUSPECTED_FRAUD|213.92 |47.99 |399.98 |15830.28 |160 |74 |
2013-08 |CANCELED |186.86 |24.99 |499.95 |59422.67 |730 |318 |
2013-08 |CLOSED |197.51 |9.99 |499.95 |314631.13 |3472 |1593 |
2013-08 |COMPLETE |197.86 |9.99 |1999.99 |907197.8 |10083 |4585 |
2013-08 |ON_HOLD |202.03 |9.99 |499.95 |155155.39 |1658 |768 |
2013-08 |PAYMENT_REVIEW |184.87 |29.98 |499.95 |25696.76 |295 |139 |
2013-08 |PENDING |198.77 |14.99 |499.95 |345071.18 |3809 |1736 |
2013-08 |PENDING_PAYMENT|201.6 |14.99 |599.99 |604406.7 |6627 |2998 |
2013-08 |PROCESSING |195.47 |9.99 |499.95 |349112.79 |3889 |1786 |
2013-08 |SUSPECTED_FRAUD|197.0 |31.98 |499.95 |67964.34 |748 |345 |
2013-09 |CANCELED |209.8 |24.99 |499.95 |67973.94 |687 |324 |
2013-09 |CLOSED |198.31 |14.99 |499.95 |338312.5 |3647 |1706 |
+----------+---------------+--------+---------+---------+----------+-----------------------+----------------------+
only showing top 20 rows

##### Problem statement-4:Get total orders for each order status peryear

In [16]:
SalesDataPerstatusPeryear = ordersdfSalesmap.groupBy(substring('order_date',0,4).alias("orderyear"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_peryear")).sort(substring('order_date',0,4).alias("orderyear"),'order_status')
SalesDataPerstatusPeryear.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPeryear", compression="none", header ='true')
SalesDataPerstatusPeryear.show(20,False)

+---------+---------------+--------+---------+---------+----------+-----------------------+---------------------+
orderyear|order_status |avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_peryear|
+---------+---------------+--------+---------+---------+----------+-----------------------+---------------------+
2013 |CANCELED |195.93 |14.99 |500.0 |314083.12 |3513 |1603 |
2013 |CLOSED |199.8 |9.99 |599.99 |1713090.03|18588 |8574 |
2013 |COMPLETE |199.34 |9.99 |1999.99 |4973802.1 |54616 |24951 |
2013 |ON_HOLD |201.02 |9.99 |1999.99 |829421.22 |9075 |4126 |
2013 |PAYMENT_REVIEW |193.34 |14.99 |499.95 |143455.81 |1594 |742 |
2013 |PENDING |199.52 |9.99 |999.99 |1715070.62|18972 |8596 |
2013 |PENDING_PAYMENT|198.33 |9.99 |599.99 |3386687.24|36995 |17076 |
2013 |PROCESSING |199.37 |9.99 |1999.99 |1839024.53|20281 |9224 |
2013 |SUSPECTED_FRAUD|195.03 |14.99 |499.95 |339554.71 |3775 |1741 |
2014 |CANCELED |199.35 |14.99 |599.99 |381947.89 |4189 |1916 |
2014 |CLOSED |200.41 |9.99 |1999.99 |2022958.83|21922 |10094 |
2014 |COMPLETE |198.28 |9.99 |1999.99 |6303131.81|69257 |31789 |
2014 |ON_HOLD |197.31 |9.99 |1999.99 |1035310.06|11339 |5247 |
2014 |PAYMENT_REVIEW |203.21 |9.99 |499.95 |214385.64 |2419 |1055 |
2014 |PENDING |199.8 |9.99 |999.99 |2136810.73|23288 |10695 |
2014 |PENDING_PAYMENT|200.19 |9.99 |999.99 |4194983.96|45940 |20955 |
2014 |PROCESSING |201.39 |9.99 |999.99 |2351612.32|25341 |11677 |
2014 |SUSPECTED_FRAUD|199.95 |15.99 |499.95 |427289.99 |4654 |2137 |
+---------+---------------+--------+---------+---------+----------+-----------------------+---------------------+

##### Problem statement-5:Get monthly sales data for each order status for first quarter of 2014

In [18]:
SalesDataPerMonth=ordersdfSalesmap.where((to_date(ordersdfSalesmap.order_date) >=lit("2014-01-01")) & 
                                          (to_date(ordersdfSalesmap.order_date) <=lit("2014-03-31")))
SalesDataPerstatusPeryear = SalesDataPerMonth.groupBy(substring('order_date',0,7).alias("ordermonth"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_permonth")).sort(substring('order_date',0,7).alias("ordermonth"),'order_status')
SalesDataPerstatusPeryear.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPeryear", compression="none", header ='true')
SalesDataPerstatusPeryear.show(20,False)

+----------+---------------+--------+---------+---------+----------+-----------------------+----------------------+
ordermonth|order_status |avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_permonth|
+----------+---------------+--------+---------+---------+----------+-----------------------+----------------------+
2014-01 |CANCELED |203.85 |19.99 |499.95 |53612.83 |591 |263 |
2014-01 |CLOSED |199.89 |14.99 |500.0 |298236.04 |3247 |1492 |
2014-01 |COMPLETE |198.04 |9.99 |1999.99 |931985.73 |9958 |4706 |
2014-01 |ON_HOLD |202.93 |14.99 |499.95 |181414.96 |1913 |894 |
2014-01 |PAYMENT_REVIEW |193.6 |9.99 |499.95 |35816.65 |404 |185 |
2014-01 |PENDING |201.33 |9.99 |499.95 |342057.67 |3818 |1699 |
2014-01 |PENDING_PAYMENT|198.65 |9.99 |499.95 |670633.09 |7324 |3376 |
2014-01 |PROCESSING |203.37 |9.99 |999.99 |346943.35 |3760 |1706 |
2014-01 |SUSPECTED_FRAUD|184.77 |24.99 |499.95 |63746.75 |754 |345 |
2014-02 |CANCELED |207.0 |22.0 |499.95 |65825.08 |667 |318 |
2014-02 |CLOSED |195.63 |9.99 |499.95 |302636.06 |3251 |1547 |
2014-02 |COMPLETE |196.09 |9.99 |500.0 |915134.06 |10251 |4667 |
2014-02 |ON_HOLD |197.15 |14.99 |500.0 |148648.08 |1642 |754 |
2014-02 |PAYMENT_REVIEW |209.45 |39.99 |499.95 |29113.01 |329 |139 |
2014-02 |PENDING |201.08 |15.99 |499.95 |316896.06 |3486 |1576 |
2014-02 |PENDING_PAYMENT|201.2 |14.99 |500.0 |612843.94 |6728 |3046 |
2014-02 |PROCESSING |193.32 |9.99 |999.99 |327870.19 |3625 |1696 |
2014-02 |SUSPECTED_FRAUD|205.15 |15.99 |499.95 |59697.24 |659 |291 |
2014-03 |CANCELED |196.87 |22.0 |599.99 |57485.89 |634 |292 |
2014-03 |CLOSED |199.26 |14.99 |499.95 |301673.84 |3344 |1514 |
+----------+---------------+--------+---------+---------+----------+-----------------------+----------------------+
only showing top 20 rows

##### Problem statement-6:Get monthly sales data for last quarter of 2013 with neither 'COMPLETE' nor 'CLOSED' orders

In [20]:
SalesDataPerMonth=ordersdfSalesmap.where((to_date(ordersdfSalesmap.order_date) >=lit("2013-10-01")) & 
                                          (to_date(ordersdfSalesmap.order_date) <=lit("2013-12-31")))

SalesDataPerstatusPermonth = SalesDataPerMonth.groupBy(substring('order_date',0,7).alias("ordermonth"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_perstatus")).\
                                                                filter((SalesDataPerMonth.order_status!='COMPLETE') & (SalesDataPerMonth.order_status!='CLOSED')).\
                                                                sort('ordermonth','order_status')
SalesDataPerstatusPermonth.coalesce(1).write.csv("/FileStore/tables/SalesDataPerstatusPermonth", compression="none", header ='true')
SalesDataPerstatusPermonth.show(20,False)

+----------+---------------+--------+---------+---------+----------+-----------------------+-----------------------+
ordermonth|order_status |avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_perstatus|
+----------+---------------+--------+---------+---------+----------+-----------------------+-----------------------+
2013-10 |CANCELED |199.32 |19.99 |499.95 |51024.94 |560 |256 |
2013-10 |ON_HOLD |196.83 |19.99 |499.95 |144669.59 |1616 |735 |
2013-10 |PAYMENT_REVIEW |189.09 |30.0 |499.95 |22879.35 |278 |121 |
2013-10 |PENDING |196.08 |15.99 |599.99 |273922.9 |3102 |1397 |
2013-10 |PENDING_PAYMENT|198.17 |9.99 |499.95 |566179.71 |6107 |2857 |
2013-10 |PROCESSING |200.09 |9.99 |499.95 |340758.39 |3814 |1703 |
2013-10 |SUSPECTED_FRAUD|193.06 |17.99 |499.95 |53478.84 |581 |277 |
2013-11 |CANCELED |197.52 |14.99 |499.95 |62812.76 |702 |318 |
2013-11 |ON_HOLD |198.86 |14.99 |499.95 |181362.63 |1990 |912 |
2013-11 |PAYMENT_REVIEW |186.94 |19.99 |499.95 |27480.01 |312 |147 |
2013-11 |PENDING |199.91 |19.98 |499.95 |364237.39 |4022 |1822 |
2013-11 |PENDING_PAYMENT|196.89 |9.99 |599.99 |721418.19 |7890 |3664 |
2013-11 |PROCESSING |202.16 |9.99 |499.95 |358435.04 |3840 |1773 |
2013-11 |SUSPECTED_FRAUD|194.6 |14.99 |499.95 |72974.71 |824 |375 |
2013-12 |CANCELED |191.39 |19.99 |500.0 |62966.39 |698 |329 |
2013-12 |ON_HOLD |206.22 |9.99 |499.95 |147652.14 |1591 |716 |
2013-12 |PAYMENT_REVIEW |192.5 |19.99 |499.95 |26757.83 |298 |139 |
2013-12 |PENDING |197.12 |9.99 |499.95 |322284.07 |3527 |1635 |
2013-12 |PENDING_PAYMENT|195.89 |9.99 |599.99 |682300.32 |7566 |3483 |
2013-12 |PROCESSING |203.54 |9.99 |1999.99 |353135.7 |3798 |1735 |
+----------+---------------+--------+---------+---------+----------+-----------------------+-----------------------+
only showing top 20 rows

###### Problem statement-7:Get monthly sales data for last quarter of 2013 and first quarter of 2014  with 'CANCELED' orders

In [22]:
SalesDataPerMonth=ordersdfSalesmap.where((to_date(ordersdfSalesmap.order_date) >=lit("2013-10-01")) & 
                                                 (to_date(ordersdfSalesmap.order_date) <=lit("2013-12-31")) |
                                                 (to_date(ordersdfSalesmap.order_date) >=lit("2014-01-01")) & 
                                                 (to_date(ordersdfSalesmap.order_date) <=lit("2014-03-31")))
SalesDatacanceledordersquarter = SalesDataPerMonth.groupBy(substring('order_date',0,7).alias("ordermonth"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_perstatus")).\
                                                                filter(SalesDataPerMonth.order_status =='CANCELED').\
                                                                sort('ordermonth','order_status')
SalesDatacanceledordersquarter.coalesce(1).write.csv("/FileStore/tables/SalesDatacanceledordersquarter", compression="none", header ='true')
SalesDatacanceledordersquarter.show(30,False)

+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+
ordermonth|order_status|avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_perstatus|
+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+
2013-10 |CANCELED |199.32 |19.99 |499.95 |51024.94 |560 |256 |
2013-11 |CANCELED |197.52 |14.99 |499.95 |62812.76 |702 |318 |
2013-12 |CANCELED |191.39 |19.99 |500.0 |62966.39 |698 |329 |
2014-01 |CANCELED |203.85 |19.99 |499.95 |53612.83 |591 |263 |
2014-02 |CANCELED |207.0 |22.0 |499.95 |65825.08 |667 |318 |
2014-03 |CANCELED |196.87 |22.0 |599.99 |57485.89 |634 |292 |
+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+

###### Problem statement-8:Get monthly sales data for 'PENDING' orders

In [24]:
SalesDatapendingorder = SalesDataPerMonth.groupBy(substring('order_date',0,7).alias("ordermonth"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_perstatus")).\
                                                                filter(SalesDataPerMonth.order_status =='PENDING').\
                                                                sort('ordermonth','order_status')
SalesDatapendingorder.coalesce(1).write.csv("/FileStore/tables/SalesDatapendingorder", compression="none", header ='true')
SalesDatapendingorder.show(30,False)

+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+
ordermonth|order_status|avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_perstatus|
+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+
2013-10 |PENDING |196.08 |15.99 |599.99 |273922.9 |3102 |1397 |
2013-11 |PENDING |199.91 |19.98 |499.95 |364237.39 |4022 |1822 |
2013-12 |PENDING |197.12 |9.99 |499.95 |322284.07 |3527 |1635 |
2014-01 |PENDING |201.33 |9.99 |499.95 |342057.67 |3818 |1699 |
2014-02 |PENDING |201.08 |15.99 |499.95 |316896.06 |3486 |1576 |
2014-03 |PENDING |201.87 |19.99 |599.99 |305423.94 |3343 |1513 |
+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+

###### Problem statement-9:Get monthly sales data for 'PROCESSING' or ON_HOLD orders

In [26]:
SalesDatacanceledordersquarter = SalesDataPerMonth.groupBy(substring('order_date',0,7).alias("ordermonth"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_perstatus")).\
                                                                filter((SalesDataPerMonth.order_status=='PROCESSING') | (SalesDataPerMonth.order_status=='ON_HOLD')).\
                                                                sort('ordermonth','order_status')
SalesDataprocessingcanceledorders.coalesce(1).write.csv("/FileStore/tables/SalesDataprocessingcanceledorders", compression="none", header ='true')
SalesDatacanceledordersquarter.show(30,False)

+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+
ordermonth|order_status|avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_perstatus|
+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+
2013-10 |ON_HOLD |196.83 |19.99 |499.95 |144669.59 |1616 |735 |
2013-10 |PROCESSING |200.09 |9.99 |499.95 |340758.39 |3814 |1703 |
2013-11 |ON_HOLD |198.86 |14.99 |499.95 |181362.63 |1990 |912 |
2013-11 |PROCESSING |202.16 |9.99 |499.95 |358435.04 |3840 |1773 |
2013-12 |ON_HOLD |206.22 |9.99 |499.95 |147652.14 |1591 |716 |
2013-12 |PROCESSING |203.54 |9.99 |1999.99 |353135.7 |3798 |1735 |
2014-01 |ON_HOLD |202.93 |14.99 |499.95 |181414.96 |1913 |894 |
2014-01 |PROCESSING |203.37 |9.99 |999.99 |346943.35 |3760 |1706 |
2014-02 |ON_HOLD |197.15 |14.99 |500.0 |148648.08 |1642 |754 |
2014-02 |PROCESSING |193.32 |9.99 |999.99 |327870.19 |3625 |1696 |
2014-03 |ON_HOLD |201.85 |14.99 |1999.99 |162486.62 |1687 |805 |
2014-03 |PROCESSING |201.24 |15.99 |599.99 |353374.54 |3821 |1756 |
+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+

###### Problem statement-10:Get monthly sales data with  'COMPLETE' AND 'CLOSED' orders

In [28]:
SalesDatacompleteandclosedorders = SalesDataPerMonth.groupBy(substring('order_date',0,7).alias("ordermonth"),'order_status').agg(round(avg('order_item_subtotal'),2).alias("avgsales"), 
                                                                min('order_item_subtotal').alias("min_sales"),
                                                                max('order_item_subtotal').alias("max_sales"),
                                                                round(sum('order_item_subtotal'),2).alias("Totalsales"),
                                                                sum('order_item_quantity').alias("Totalquantity_perstatus"),
                                                                count('order_item_subtotal').alias("Numberofsales_perstatus")).\
                                                                filter((SalesDataPerMonth.order_status=='COMPLETE') | (SalesDataPerMonth.order_status=='CLOSED')).\
                                                                sort('ordermonth','order_status')
SalesDatacompleteandclosedorders.coalesce(1).write.csv("/FileStore/tables/SalesDatacompleteandclosedorders", compression="none", header ='true')
SalesDatacompleteandclosedorders.show(30,False)

+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+
ordermonth|order_status|avgsales|min_sales|max_sales|Totalsales|Totalquantity_perstatus|Numberofsales_perstatus|
+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+
2013-10 |CLOSED |200.55 |9.99 |599.99 |294414.07 |3184 |1468 |
2013-10 |COMPLETE |199.29 |9.99 |1999.99 |877272.88 |9662 |4402 |
2013-11 |CLOSED |200.57 |14.99 |499.95 |339567.76 |3725 |1693 |
2013-11 |COMPLETE |200.03 |9.99 |499.95 |1040367.6 |11400 |5201 |
2013-12 |CLOSED |200.96 |9.99 |599.99 |348057.15 |3727 |1732 |
2013-12 |COMPLETE |200.1 |9.99 |500.0 |929662.48 |10055 |4646 |
2014-01 |CLOSED |199.89 |14.99 |500.0 |298236.04 |3247 |1492 |
2014-01 |COMPLETE |198.04 |9.99 |1999.99 |931985.73 |9958 |4706 |
2014-02 |CLOSED |195.63 |9.99 |499.95 |302636.06 |3251 |1547 |
2014-02 |COMPLETE |196.09 |9.99 |500.0 |915134.06 |10251 |4667 |
2014-03 |CLOSED |199.26 |14.99 |499.95 |301673.84 |3344 |1514 |
2014-03 |COMPLETE |198.7 |14.99 |1999.99 |969677.16 |10642 |4880 |
+----------+------------+--------+---------+---------+----------+-----------------------+-----------------------+